# RAG Pipeline with Opik Tracking

This notebook demonstrates how to integrate Opik for:
- Prompt tracking and versioning
- LLM call logging
- RAG pipeline monitoring
- Experiment tracking

## Setup

In [1]:
# Install Opik if not already installed
# !pip install opik

In [2]:
import sys
from pathlib import Path

project_root = str(Path.cwd().parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
from abc import ABC, abstractmethod
from typing import List
import ollama
from src.utils.config import LLMConfig, settings
from src.shared.models import SearchResult

# Opik imports
import opik
from opik import track

## Initialize Opik

Configure Opik to track your experiments. You can use the cloud version or self-hosted.

In [4]:
# Initialize Opik client
# For cloud: opik.configure(api_key="your-api-key")
# For local: opik.configure(use_local=True)

opik_client = opik.Opik()

# Optional: Create or use existing project
PROJECT_NAME = "rag-pipeline"
print(f"✅ Opik initialized - Project: {PROJECT_NAME}")

OPIK: ========================
The API key must be specified to log data to https://www.comet.com/opik.
You can use `opik configure` CLI command to configure your environment for logging.
See the configuration details in the docs: https://www.comet.com/docs/opik/tracing/sdk_configuration.





✅ Opik initialized - Project: rag-pipeline


## Prompt Management with Opik

Store and version your prompts using Opik's prompt management features.

In [5]:
# Create prompt templates with versioning
from opik import prompt_template

# Query refinement prompt (v1)
QUERY_REFINEMENT_PROMPT_V1 = prompt_template(
    name="query_refinement",
    version="v1",
    template="""Generate 3 different versions of this question for better document search.
Original question: {question}

Provide 3 alternative phrasings, one per line:"""
)

# Answer generation prompt (v1)
ANSWER_GENERATION_PROMPT_V1 = prompt_template(
    name="answer_generation",
    version="v1",
    template="""Answer this question using only the context below.

Context:
{context}

Question: {question}

Answer:"""
)

print("✅ Prompt templates registered with Opik")

ImportError: cannot import name 'prompt_template' from 'opik' (/home/moad/desktop/open-books/.venv/lib/python3.11/site-packages/opik/__init__.py)

## Ollama Manager

In [ ]:
class OllamaManager:
    @staticmethod
    def ensure_ready(model_name: str = "llama3.2"):
        try:
            models = ollama.list()
            if not any(model_name in m['name'] for m in models['models']):
                print(f" Downloading {model_name}...")
                ollama.pull(model_name)
            print(f" {model_name} ready")
            return True
        except Exception as e:
            print(f"  Ollama not running!")
            print(f"   Run: ollama serve")
            print(f"   Then: ollama pull {model_name}")
            return False

## Generator with Opik Tracking

In [ ]:
class BaseGenerator(ABC):
    @abstractmethod
    def generate(self, prompt: str) -> str:
        pass

class OllamaGenerator(BaseGenerator):
    def __init__(self, config: LLMConfig, auto_setup: bool = True):
        self.model = config.model_name
        self.temperature = config.temperature
        if auto_setup:
            OllamaManager.ensure_ready(self.model)
    
    @track(name="ollama_generation", project_name=PROJECT_NAME)
    def generate(self, prompt: str, metadata: dict = None) -> str:
        """Generate response with Opik tracking.
        
        Args:
            prompt: The input prompt
            metadata: Additional metadata to log (e.g., prompt version, purpose)
        """
        try:
            # Track the LLM call
            response = ollama.chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={"temperature": self.temperature}
            )
            
            result = response['message']['content']
            
            # Log additional metadata if provided
            if metadata:
                opik.track_metadata(metadata)
            
            # Log input/output
            opik.log_traces(
                input={"prompt": prompt, "metadata": metadata},
                output={"response": result},
                tags=["generation", self.model]
            )
            
            return result
        except Exception as e:
            opik.log_error(str(e))
            return f"Error: {e}"

## Query Constructor with Prompt Versioning

In [ ]:
class QueryConstructor(ABC):
    @abstractmethod
    def refine_query(self, query: str) -> list[str]:
        pass

class MultiQueryConstructor(QueryConstructor):
    def __init__(self, generator: BaseGenerator, prompt_version: str = "v1") -> None:
        self.generator = generator
        self.prompt_version = prompt_version
        # Use versioned prompt template
        self.template = QUERY_REFINEMENT_PROMPT_V1

    @track(name="query_refinement", project_name=PROJECT_NAME)
    def refine_query(self, query: str) -> list[str]:
        """Refine query with tracking."""
        # Format prompt using template
        prompt = self.template.format(question=query)
        
        # Generate with metadata about prompt version
        metadata = {
            "prompt_version": self.prompt_version,
            "prompt_name": "query_refinement",
            "original_query": query
        }
        
        response = self.generator.generate(prompt, metadata=metadata)
        queries = [q.strip() for q in response.split('\n') if q.strip() and len(q.strip()) > 10]
        
        refined_queries = [query] + queries[:3]
        
        # Log the refinement results
        opik.log_traces(
            input={"original_query": query},
            output={"refined_queries": refined_queries, "count": len(refined_queries)},
            tags=["query_refinement", self.prompt_version]
        )
        
        return refined_queries

## Answer Generator with Tracking

In [ ]:
class BaseQueryAnswerer(ABC):
    @abstractmethod
    def answer(self, result_search: List[SearchResult], query: str) -> str:
        pass

class QueryAnswerer(BaseQueryAnswerer):
    def __init__(self, generator: BaseGenerator, prompt_version: str = "v1") -> None:
        self.generator = generator
        self.prompt_version = prompt_version
        # Use versioned prompt template
        self.template = ANSWER_GENERATION_PROMPT_V1
    
    @track(name="answer_generation", project_name=PROJECT_NAME)
    def answer(self, result_search: List[SearchResult], query: str) -> str:
        """Generate answer with tracking."""
        if not result_search:
            return "No relevant documents found."
        
        # Prepare context
        context_parts = [f"[{i}] {r.content}" for i, r in enumerate(result_search, 1)]
        context = "\n\n".join(context_parts)
        
        # Format prompt
        prompt = self.template.format(context=context, question=query)
        
        # Generate with metadata
        metadata = {
            "prompt_version": self.prompt_version,
            "prompt_name": "answer_generation",
            "num_context_chunks": len(result_search),
            "context_sources": [r.metadata.get('source', 'unknown') for r in result_search]
        }
        
        answer = self.generator.generate(prompt, metadata=metadata)
        
        # Log the answer generation
        opik.log_traces(
            input={
                "query": query,
                "num_chunks": len(result_search)
            },
            output={"answer": answer},
            tags=["answer_generation", self.prompt_version]
        )
        
        return answer

## RAG Pipeline with Full Tracking

In [ ]:
class SimpleRAGPipeline:
    def __init__(self, vector_store, answerer: BaseQueryAnswerer):
        self.vector_store = vector_store
        self.answerer = answerer
    
    @track(name="simple_rag_query", project_name=PROJECT_NAME)
    def query(self, query: str, top_k: int = 5) -> str:
        """Execute RAG query with tracking."""
        # Retrieve documents
        results = self.vector_store.query([query], n_result=top_k)
        
        # Log retrieval
        opik.log_traces(
            input={"query": query, "top_k": top_k},
            output={"num_results": len(results)},
            tags=["retrieval", "simple_rag"]
        )
        
        # Generate answer
        answer = self.answerer.answer(results, query)
        
        return answer


class MultiQueryRAGPipeline:
    def __init__(self, query_constructor: QueryConstructor, vector_store, answerer: BaseQueryAnswerer):
        self.query_constructor = query_constructor
        self.vector_store = vector_store
        self.answerer = answerer
    
    @track(name="multi_query_rag", project_name=PROJECT_NAME)
    def query(self, query: str, top_k: int = 5) -> str:
        """Execute multi-query RAG with full tracking."""
        # Refine query (tracked in query_constructor)
        refined_queries = self.query_constructor.refine_query(query)
        print(f" Using {len(refined_queries)} query variations")
        
        # Retrieve with all queries
        all_results = self.vector_store.query(refined_queries, n_result=top_k)
        print(f" Found {len(all_results)} total results")
        
        # Log retrieval metrics
        opik.log_traces(
            input={
                "original_query": query,
                "refined_queries": refined_queries,
                "top_k": top_k
            },
            output={
                "num_results": len(all_results),
                "num_queries_used": len(refined_queries)
            },
            tags=["retrieval", "multi_query_rag"]
        )
        
        # Generate answer (tracked in answerer)
        answer = self.answerer.answer(all_results, query)
        
        return answer

## Initialize Components

In [ ]:
# Create generator
config = LLMConfig(model_name="llama3.2", temperature=0.7)
generator = OllamaGenerator(config)

# Create answerer with prompt versioning
answerer = QueryAnswerer(generator, prompt_version="v1")

print("✅ Components initialized with Opik tracking")

## Load Vector Store

*(Use your existing vector store loading code)*

In [ ]:
# Your existing vector store code here
# from src.ingestion.vector_store import ...
# vector_store = ...

## Test with Tracking

In [ ]:
# Test simple RAG
simple_rag = SimpleRAGPipeline(vector_store=vector_store, answerer=answerer)

# This will be tracked in Opik
answer = simple_rag.query("What is Word2Vec?", top_k=3)
print("\n📝 Answer:")
print(answer)

In [ ]:
# Test multi-query RAG
query_constructor = MultiQueryConstructor(generator, prompt_version="v1")
multi_rag = MultiQueryRAGPipeline(
    query_constructor=query_constructor,
    vector_store=vector_store,
    answerer=answerer
)

# This will track the entire pipeline
answer = multi_rag.query("What is Word2Vec?", top_k=5)
print("\n📝 Answer:")
print(answer)

## Experiment Tracking

Run experiments with different prompt versions and compare results.

In [ ]:
# Create a new prompt version for testing
ANSWER_GENERATION_PROMPT_V2 = prompt_template(
    name="answer_generation",
    version="v2",
    template="""You are a helpful assistant. Answer the question using the context provided.
Be concise and cite sources when possible.

Context:
{context}

Question: {question}

Detailed Answer:"""
)

# Test both versions
test_queries = [
    "What is Word2Vec?",
    "How does Word2Vec work?",
    "What are the applications of word embeddings?"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    # Test with v1
    answerer_v1 = QueryAnswerer(generator, prompt_version="v1")
    rag_v1 = SimpleRAGPipeline(vector_store=vector_store, answerer=answerer_v1)
    answer_v1 = rag_v1.query(query, top_k=3)
    
    print(f"\nV1 Answer: {answer_v1[:200]}...")
    
    # Note: You would create answerer_v2 with the new template
    # This demonstrates how to track different versions

## View Results in Opik Dashboard

After running your experiments, you can:

1. **View traces**: See the full execution flow of each RAG query
2. **Compare prompts**: Compare different versions side-by-side
3. **Analyze performance**: Track latency, token usage, costs
4. **Debug issues**: Inspect failed queries and errors
5. **Track experiments**: Compare different configurations

Access the dashboard at:
- Cloud: https://www.comet.com/site/products/opik/
- Local: http://localhost:5173 (if using local deployment)

## Advanced: Custom Metrics and Feedback

Add custom metrics to track quality.

In [ ]:
from opik import score

def evaluate_answer_quality(query: str, answer: str, expected_keywords: List[str]) -> float:
    """Simple keyword-based quality score."""
    found_keywords = sum(1 for kw in expected_keywords if kw.lower() in answer.lower())
    return found_keywords / len(expected_keywords) if expected_keywords else 0.0

# Example usage
query = "What is Word2Vec?"
answer = simple_rag.query(query, top_k=3)

# Score the answer
quality_score = evaluate_answer_quality(
    query=query,
    answer=answer,
    expected_keywords=["vector", "word", "embedding", "neural"]
)

# Log the score
opik.log_metric(
    name="answer_quality",
    value=quality_score,
    tags=["evaluation", "keyword_match"]
)

print(f"Quality Score: {quality_score:.2f}")

## Feedback Loop

Collect user feedback on answers.

In [ ]:
def collect_feedback(trace_id: str, rating: int, comment: str = ""):
    """Collect user feedback on a specific answer."""
    opik.log_feedback(
        trace_id=trace_id,
        score=rating,
        comment=comment
    )

# Example: After getting an answer, collect feedback
# trace_id = opik.get_current_trace_id()
# collect_feedback(trace_id, rating=4, comment="Good answer but could be more detailed")